In [32]:
# import necessary packages
import os
import pandas
import numpy
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
# import function for querying DOE dataframes
from monte_carlo_utils import get_DOE_atb, get_DOE_barges

# set file location and name
atb_shp_path = Path('/data/MIDOSS/shapefiles/atb_2018_01.shp')
barge_shp_path = Path('/data/MIDOSS/shapefiles/barge_2018_01.shp')
doe_xls_path = Path('/data/MIDOSS/spreadsheets/MuellerTrans4-30-20.xlsx')
fac_xls_path = Path(
    '/home/rmueller/Projects/MIDOSS/marine_transport_data/Oil_Transfer_Facilities.xlsx'
)

# import facility locations
facWA = pandas.read_excel(
    fac_xls_path,
    sheet_name = 'Washington',
    usecols="B,D,J,K"
)

# import shapefile using geopandas
allTracks = {} 
allTracks['atb']= gpd.read_file(atb_shp_path)
allTracks['barge'] = gpd.read_file(barge_shp_path)

# view  the top 6 lines of attribute table of data
allTracks['atb'].head(1)

,MMSI_NUM,ST_DATE,EN_DATE,LENGTH,TO,FROM_,geometry
0,316001223.0,2018-01-01 00:03:48,2018-01-01 00:05:58,41,US,US,"LINESTRING (-122.77574 48.33663, -122.77796 48..."


In [38]:
attribution = ['US','Canada','Pacific']
noNone = {}
allNone = {}
generic = {}

for vessel_type in ["atb",'barge']:
    generic[vessel_type] = allTracks[vessel_type].loc[
        allTracks[vessel_type].TO.isin(attribution) & 
        allTracks[vessel_type].FROM_.isin(attribution)
    ]

In [3]:
generic["atb"].head(1)

,MMSI_NUM,ST_DATE,EN_DATE,LENGTH,TO,FROM_,geometry
0,316001223.0,2018-01-01 00:03:48,2018-01-01 00:05:58,41,US,US,"LINESTRING (-122.77574 48.33663, -122.77796 48..."


## ATB

In [4]:
# drop rows with None attribution
noNone['atb'] = allTracks['atb'].dropna().reset_index(drop=True)
# keep rows with None attribution are missing
shp_tmp = allTracks['atb'].isnull()
row_has_None = shp_tmp.any(axis=1)
allNone['atb'] = allTracks['atb'][row_has_None]
#print results
print(allTracks['atb'].shape[0])
print(noNone['atb'].shape[0])
print(allNone['atb'].shape[0])
print(f'None + NoNone = {allNone["atb"].shape[0] + noNone["atb"].shape[0]}')

27221
16324
10897
None + NoNone = 27221


In [5]:
print(f'Fraction of ATB tracks that are unattributed: {allNone["atb"].shape[0]/allTracks["atb"].shape[0]}')
print(f'Fraction of ATB tracks that are attributed: {noNone["atb"].shape[0]/allTracks["atb"].shape[0]}')
print(f'Fraction of ATB tracks that are generic attribution: {generic["atb"].shape[0]/allTracks["atb"].shape[0]}')

Fraction of ATB tracks that are unattributed: 0.40031593255207376
Fraction of ATB tracks that are attributed: 0.5996840674479262
Fraction of ATB tracks that are generic attribution: 0.3468278167591198


## Barges

In [8]:
vessel_type = "barge"
# drop rows with None attribution
noNone[vessel_type] = allTracks[vessel_type].dropna().reset_index(drop=True)
# keep rows with None attribution are missing
shp_tmp = allTracks[vessel_type].isnull()
row_has_None = shp_tmp.any(axis=1)
allNone[vessel_type] = allTracks[vessel_type][row_has_None]
#print results
print(allTracks[vessel_type].shape[0])
print(noNone[vessel_type].shape[0])
print(allNone[vessel_type].shape[0])
print(f'None + NoNone = {allNone[vessel_type].shape[0] + noNone[vessel_type].shape[0]}')
print(f'Fraction of {vessel_type} tracks that are unattributed: {allNone[vessel_type].shape[0]/allTracks[vessel_type].shape[0]}')
print(f'Fraction of {vessel_type} tracks that are attributed: {noNone[vessel_type].shape[0]/allTracks[vessel_type].shape[0]}')
print(f'Fraction of {vessel_type} tracks that are generic attribution: {generic[vessel_type].shape[0]/allTracks[vessel_type].shape[0]}')

624426
372124
252302
None + NoNone = 624426
Fraction of barge tracks that are unattributed: 0.4040542834539241
Fraction of barge tracks that are attributed: 0.5959457165460759
Fraction of barge tracks that are generic attribution: 0.5009192442339044


## Quantify barge and ATB cargo transfers in 2018 DOE database

In [16]:
[atb_in, atb_out]=get_DOE_atb(
    doe_xls_path, 
    fac_xls_path, 
    transfer_type = 'cargo', 
    facilities='selected'
)

barge_inout=get_DOE_barges(
    doe_xls_path,
    fac_xls_path, 
    direction='combined',
    facilities='selected',
    transfer_type = 'cargo')

get_DOE_barges: not yet tested with fac_xls as input
combined
cargo
cargo


In [18]:
transfers = {}
transfers["barge"] = barge_inout.shape[0]
transfers["atb"] = atb_in.shape[0] + atb_out.shape[0]
print(f'{transfers["atb"]} cargo transfers for atbs')
print(f'{transfers["barge"]} cargo transfers for barges')

677 cargo transfers for atbs
2773 cargo transfers for barges


### Calculate the number of Attributed tracks we get for ATBs and estimate the equivalent value for barges

In [21]:
noNone["atb"].shape[0]

16324

In [31]:
# estimate the ratio of attributed ATB tracks to ATB cargo transfers
noNone_ratio = noNone["atb"].shape[0]/transfers["atb"]
print(f'We get {noNone_ratio:.2f} attributed ATB tracks per ATB cargo transfer')
# estimate the amount of attributed tracks we'd expect to see for tank barges based on tank barge transfers
print(f'We expect {noNone_ratio*transfers["barge"]:.2f} attributed barge tracks, but we get {noNone["barge"].shape[0]}')
# estimate spurious barge voyages by removing estimated oil carge barge from total
fraction_nonoilbarge = (noNone["barge"].shape[0]-noNone_ratio*transfers["barge"])/noNone["barge"].shape[0]
print(f'We estimate that non-oil tank barge voyages account for {100*fraction_nonoilbarge:.2f}% of barge voyages')

We get 24.11 attributed ATB tracks per ATB cargo transfer
We expect 66863.30 attributed barge tracks, but we get 372124
We estimate that non-oil tank barge voyages account for 82.03% of barge voyages


## Now look more carefully at marine terminal attributions to get a sense of how much oil cargo tug traffic "moonlights" with other non-oil gigs.  

In [33]:
facWA.head()

,FacilityName,FacilityDOEName,DockLatNumber,DockLongNumber
0,BP Cherry Point Refinery,BP Cherry Point Refinery,48.86111,-122.758
1,Shell Puget Sound Refinery,Shell Puget Sound Refinery,48.50945,-122.577
2,Shell Oil LP Seattle Distribution Terminal,Shell Oil LP Seattle Distribution Terminal,47.58870,-122.353
3,Maxum Petroleum - Harbor Island Terminal,Maxum (Rainer Petroleum),47.58753,-122.353
4,Tidewater Snake River Terminal,Tidewater Snake River Terminal,46.22312,-119.014


In [35]:
allfac = {}
for vessel_type in ["atb",'barge']:
    allfac[vessel_type] = allTracks[vessel_type].loc[
        (allTracks[vessel_type].TO.isin(facWA.FacilityName) |
         allTracks[vessel_type].FROM_.isin(facWA.FacilityName))
    ] 

In [36]:
print(f'{allfac["atb"].shape[0]} ATB tracks have a WA oil facility as origin or destination')
print(f'{allfac["barge"].shape[0]} barge tracks have a WA oil facility as origin or destination')
fac_att_ratio = allfac["atb"].shape[0]/transfers["atb"]
print(f'We get {fac_att_ratio:.2f} WA oil marine terminal attributed ATB tracks per ATB cargo transfer')
# estimate the amount of oil cargo facility tracks we'd expect to see for tank barges based on tank barge transfers
print(f'We expect {fac_att_ratio*transfers["barge"]:.2f} WA oil marine terminal attributed barge tracks, but we get {allfac["barge"].shape[0]}')
fraction_nonoilbarge = (allfac["barge"].shape[0]-fac_att_ratio*transfers["barge"])/allfac["barge"].shape[0]
print(f'We estimate that non-oil tank barge voyages to/from marine terminals account for {100*fraction_nonoilbarge:.2f}% of barge voyages attributed to WA marine terminals')


5345 ATB tracks have a WA oil facility as origin or destination
87516 barge tracks have a WA oil facility as origin or destination
We get 7.90 WA oil marine terminal attributed ATB tracks per ATB cargo transfer
We expect 21893.18 WA oil marine terminal attributed barge tracks, but we get 87516
We estimate that non-oil tank barge voyages to/from marine terminals account for 74.98% of barge voyages attributed to WA marine terminals


## Repeat for Generic attibution only

In [40]:
print(f'{generic["atb"].shape[0]} ATB tracks have Pacific, US or Canada as origin or destination')
print(f'{generic["barge"].shape[0]} barge tracks have a WA oil facility as origin or destination')
generic_ratio = generic["atb"].shape[0]/transfers["atb"]
print(f'We get {generic_ratio:.2f} WA oil marine terminal attributed ATB tracks per ATB cargo transfer')
# estimate the amount of oil cargo facility tracks we'd expect to see for tank barges based on tank barge transfers
print(f'We expect {generic_ratio*transfers["barge"]:.2f} WA oil marine terminal attributed barge tracks, but we get {generic["barge"].shape[0]}')
fraction_nonoilbarge = (generic["barge"].shape[0]-generic_ratio*transfers["barge"])/generic["barge"].shape[0]
print(f'We estimate that non-oil tank barge voyages account for {100*fraction_nonoilbarge:.2f}% of barge voyages with both to/from as generic attributions ')



5937 ATB tracks have Pacific, US or Canada as origin or destination
269229 barge tracks have a WA oil facility as origin or destination
We get 8.77 WA oil marine terminal attributed ATB tracks per ATB cargo transfer
We expect 24318.02 WA oil marine terminal attributed barge tracks, but we get 269229
We estimate that non-oil tank barge voyages account for 90.97% of barge voyages with both to/from as generic attributions 
